In [65]:
import torch
torch.set_float32_matmul_precision('high')
# torch._dynamo.config.suppress_errors = True
import math
from torch.nn import functional as F
import torch.nn as nn
import os
from nltk.tokenize import RegexpTokenizer
torch.cuda.is_available()
import re
import string
import unicodedata, itertools, sys
import spacy
from spacy import displacy
nlp = spacy.load("fr_dep_news_trf")


In [66]:
batch_size = 2 # how many independent sequences will we process in parallel?
block_size = 256  # what is the maximum context length for predictions?


learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'


n_embd = 128
n_head = 16
n_layer = 16

torch.manual_seed(445)
print(device)

cuda


In [67]:
def split_words_by_subwords(words):
    split_list = []
    new_splits = []

    for word in re.split(r"(\ |\.|\,|\   )", words):

        prefix = None
        suffix = None

        if word == '\n' or word == '\t' or word == '\r':
            continue
        word = word.lower()
        m = re.match(r'(?:\{[^-#{}/]+\})?(?:([^-#{}/]+)-)?([^-#{}/]+)(?:/[^-#{}/]+)?(?:#([^-#{}/]+))?', word)
        if m:
            prefix, root, suffix = m.groups()
            if prefix and root and suffix:
                split_list.append(prefix)
                split_list.append(root)
                split_list.append(suffix)
            elif prefix and root and not suffix:
                split_list.append(prefix)
                split_list.append(root)
            elif not prefix and root and suffix:
                split_list.append(root)
                split_list.append(suffix)
            elif not prefix and root and not suffix:
                split_list.append(root)
    return split_list

In [68]:


# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('contes_de_fées.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# chars = sorted(list(set(text)))
# print(chars)
# vocab_size = len(chars)
# print(''.join(chars))
# print(vocab_size)


# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


# def encode(s):
#     # Tokenize the input string
#     s_tokens = tokenizer.tokenize(s)
#     # Encode only if the token exists in the stoi mapping
#     return [stoi[token] for token in s_tokens if token in stoi]

# def decode(l):
#     # Decode using the itos mapping
#     return ''.join([itos[i] for i in l if i in itos])

prefix_voc = ['anthropie', 'centrique', 'graphique', 'ificateur', 'ification', 'thermique', 'anthropo', 'cleidoco', 'déshydro', 'hollando', 
'anthrope', 'céphalie', 'métrique', 'arachno', 'arrière', 'carcino', 'duodéno', 'électro', 'germano', 'géronto', 'helleno', 'hispano', 
'hystéro', 'magnéto', 'métallo', 'myrméco', 'norvégo', 'phospho', 'spectro', 'strepto', 'thanato', 'tridéca', 'assions', 'céphale', 
'culteur', 'dactyle', 'ectomie', 'édrique', 'eraient', 'ergique', 'génique', 'graphie', 'illiard', 'isation', 'lécithe', 'logique', 
'logiste', 'plastie', 'trophie', 'actino', 'aranéo', 'archéo', 'aristo', 'balano', 'biblio', 'canado', 'cardio', 'chimio', 'chloro', 'chromo', 'chrono', 'contra', 'contre', 'crypto', 'dendro', 'dextro', 'dodéca', 'dynamo', 'échino', 'égypto', 'franco', 'gastro', 'génito', 'hémato', 'hépato', 'hétéro', 'ichtyo', 'immuno', 'ischio', 'islamo', 'japano', 'libano', 'malaco', 'maroco', 'mégalo', 'mélano', 'méthyl', 'morpho', 'musico', 'odonto', 'onycho', 'organo', 'physio', 'pneumo', 'pseudo', 'psycho', 'quadri', 'schizo', 'scytho', 'séléno', 'sesqui', 'stéréo', 'techno', 'thermo', 'adique', 'ailler', 'aillon', 'amment', 'archie', 'assent', 'assiez', 'atrice', 'chorie', 'chrome', 'claste', 'cratie', 'dingue', 'elette', 'eresse', 'erolle', 'genèse', 'glosse', 'gramme', 'graphe', 'illion', 'issime', 'lâtrie', 'lingue', 'mancie', 'métrie', 'mnésie', 'morphe', 'odonte', 'pathie', 'phagie', 'philie', 'phobie', 'scopie', 'sphère', 'thèque', 'trophe', 'unième', 'viller', 'willer', 'acéto', 'adéno', 'alter', 'amino', 'amphi', 'andro', 'anémo', 'angio', 'anglo', 'aniso', 'apico', 'archi', 'astro', 'audio', 'belgo', 'belle', 'brady', 'bucco', 'caryo', 'centi', 'cosmo', 'cyano', 'cyber', 'cyclo', 'entre', 'éroto', 'ethno', 'extra', 'femto', 'ferro', 'finno', 'frigo', 'génio', 'glyco', 'gréco', 'hagio', 'hecto', 'hélio', 'hepta', 'hippo', 'histo', 'homéo', 'hydro', 'hygro', 'hyper', 'iatro', 'infra', 'inter', 'intra', 'italo', 'judéo', 'lacto', 'libro', 'litho', 'livro', 'macro', 'métro', 'micro', 'milli', 'mnémo', 'multi', 'myélo', 'myria', 'myrio', 'mytho', 'narco', 'nécro', 'neuro', 'nitro', 'ophio', 'ortho', 'ostéo', 'paléo', 'penta', 'pétro', 'phono', 'photo', 'phyto', 'piézo', 'pluri', 'proto', 'pseud', 'ptéro', 'quasi', 'quint', 'radio', 'rétro', 'rhino', 'russo', 'socio', 'stégo', 'super', 'supra', 'tachy', 'tétra', 'trans', 'tribo', 'turco', 'ultra', 'vibro', 'vidéo', 'yotta', 'acées', 'aient', 'aille', 'aines', 'aison', 'antes', 'asses', 'ateur', 'ation', 'bourg', 'carpe', 'coque', 'crate', 'drome', 'elles', 'eraie', 
'erais', 'erait', 'èrent', 'erole', 'erons', 'eront', 'esque', 'forme', 'gamie', 'garou', 'génie', 'gonie', 'ienne', 'ifier', 'itude', 'lâtre', 'lecte', 'lithe', 'logie', 'logue', 'manie', 'mètre', 'muche', 'naute', 'nomie', 'oient', 'oïque', 'oison', 'onium', 'onyme', 'osité', 'pathe', 'pause', 'phage', 'phile', 'phone', 'phore', 'ptère', 'ptile', 'rrhée', 'saure', 'scope', 'srice', 'tomie', 'trice', 'ville', 'woman', 'aéro', 'afro', 'agri', 'agro', 'allo', 'ambi', 'anté', 'anti', 'atmo', 'atto', 'auto', 'baro', 'beau', 'caco', 'cata', 'céto', 'ciné', 'cryo', 'cyto', 'dano', 'déca', 'déci', 'demi', 'démo', 'dors', 'ecto', 'endo', 'entr', 'équi', 'euro', 'giga', 'gyro', 'héli', 'hémi', 'hémo', 'hexa', 'holo', 'homo', 'hydr', 'hypo', 'idéo', 'igni', 'indo', 'kilo', 'lact', 'Lal.', 'luso', 'méga', 'mélo', 'méno', 'méro', 'méso', 'méta', 'mini', 'Mme.', 'mono', 'myco', 'nano', 'nona', 'octo', 'oléo', 'omni', 'para', 'pédo', 'péri', 'peta', 'péta', 'pico', 'poly', 'post', 'ptil', 'pyro', 'semi', 'séro', 'sino', 'sous', 'syro', 'taxi', 'taxo', 'télé', 'téra', 'théo', 'xéno', 'xéro', 'able', 'acée', 'aine', 'aire', 'ales', 'âmes', 'ance', 'ange', 'ante', 'ants', 'arde', 'asse', 'âtes', 'atif', 'âtre', 'cèle', 'cide', 'cole', 'cyte', 'eaux', 'èdre', 'elet', 'elle', 'émie', 'ence', 'erai', 'erez', 'erie', 'eron', 'esse', 'ette', 'euse', 'fère', 'fier', 'game', 'gate', 'gène', 'gone', 'ible', 'iche', 'ième', 'ière',
 'ions', 'ique', 'iser', 'isme', 'ïsme', 'isse', 'iste', 'îtes', 'kini', 'lyse', 'mane', 'ment', 'mère', 'mont', 'nôme', 'oche', 'oïde',
  'oire', 'oter', 'otte', 'ouse', 'ouze', 'oyer', 'pole', 'punk', 'quin', 'rice', 'tion', 'ueux', 'vore', 'wihr', 'ana', 'apo', 'bio', 'con', 
  'dés', 'dia', 'dis', 'dys', 'éco', 'éka', 'épi', 'exo', 'exa', 'exo', 'for', 'géo', 'iso', 'mal', 'més', 'néo', 'œno', 'oro', 'oxy', 'pan', 
  'par', 'per', 'pré', 'pro', 'sou', 'sub', 'sur', 'sus', 'sym', 'syn', 'tri', 'uni', 'uro', 'zoo']

suffix_voc =  ['acé', 'ade', 'age', 'aie', 'ail', 'ain',
   'ais', 'ait', 'ale', 'ane', 'ant', 'ard', 'ate', 'aud', 'aux', 'eau', 'els', 'ème', 'ène', 'ent', 'era', 'eté', 'eur', 'eux', 'ide', 'idé', 
   'iel', 'ien', 'ier', 
 'iez', 'ine', 'ing', 'ion', 'ise', 'ite', 'ité', 'ive', 'man', 'oir', 'ois', 'ole', 'ome', 'ons', 'ose', 'ote', 'ule', 'ure',
  'yne', 'an', 'bi', 'dé', 'di', 'em', 'en', 'il', 'im', 'in', 'ir', 'is', 'mé', 'mi', 'né', 're', 'ré', 'ai', 'al', 
  'at', 'ât', 'ax', 'ée', 'el', 'er', 'es', 'et', 'ez', 'ie', 'if', 'in', 'ir', 'is', 'it', 'ît', 'me', 'ol', 'on', 'os', 'ot', 're',
  'té']


# processed_vocabulary = split_words_by_subwords(text)
# cleaned_list = [ x for x in processed_vocabulary if x != '' ]
# print(cleaned_list)
# cleaned_list = list(dict.fromkeys(cleaned_list))


# stoi = {ch: i for i, ch in enumerate(cleaned_list)}
# itos = {i: ch for i, ch in enumerate(cleaned_list)}


In [69]:
# def encode(text):
#     s_tokens = split_words_by_subwords(text)
#     cleaned_list = [ x for x in s_tokens if x != '' ]
#     return [stoi[token] for token in cleaned_list if token in stoi]


# def decode(l):
#     return ''.join([itos[i] for i in l if i in itos])


# toknizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
# tokenized_text = toknizer.tokenize(text)

# stoi = {ch: i for i, ch in enumerate(tokenized_text)}
# itos = {i: ch for i, ch in enumerate(tokenized_text)}

# vocab_size = len(tokenized_text)
# print(vocab_size)

# def encode(s):
#     s_tokens = toknizer.tokenize(s)
#     return [stoi[token] for token in s_tokens if token in stoi]

# def decode(l):
#     return ''.join([itos[i] for i in l if i in itos])

pattern = r"[dnl]['´`]|\w+|$[\d.]+|\S+"
tokenizer = RegexpTokenizer(pattern)

tokens = tokenizer.tokenize(text)
stoi = {ch: i for i, ch in enumerate(tokens)}
itos = {i: ch for i, ch in enumerate(tokens)}

vocab_size = len(tokens)

def encode(s):
    s_tokens = tokenizer.tokenize(s)
    return [stoi[token] for token in s_tokens if token in stoi]

def decode(l):
    return ' '.join([itos[i] for i in l if i in itos])


In [70]:
txt = """« Je sais pourquoi tu es ici !  » ?"""
encoded_text = encode(txt)
print(encoded_text)

[196364, 194243, 194188, 195172, 196178, 196131, 195108, 196333, 196375, 195342]


In [71]:
print(decode(encoded_text))

« Je sais pourquoi tu es ici ! » ?


In [72]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [73]:
#decode(train_data[0:10].tolist())

In [74]:
# # data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    if len(data) <= block_size:
        raise ValueError("Data length is too short for the model's block size.")
    if batch_size > block_size:
        raise ValueError("Batch size is larger than the model's block size.")
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.zeros((batch_size, block_size), dtype=torch.long)
    Y = torch.zeros((batch_size, block_size), dtype=torch.long)
    for i in range(batch_size):
        block_start = ix[i]
        block_end = block_start + block_size + 1
        block = data[block_start:block_end]
        X[i, :] = block[:-1]
        Y[i, :] = block[1:]
        X = X.to(device)
        Y = Y.to(device)
    return X, Y

In [75]:
def new_gelu(x):
    return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

In [76]:
# def precompute_freqs_cis(head_dim, seq_len, device, theta = 10000.0):
#     assert  head_dim % 2 == 0 # head_dim must be divisible by 2
#     theta_numerator = torch.arange(0, head_dim, 2, dtype=torch.float, device=device).float()
#     theta = 1.0 / (theta ** (theta_numerator / head_dim)).to(device)
#     m = torch.arange(0, seq_len, dtype=torch.float, device=device)
#     freqs = torch.outer(m, theta).float()
#     freqs_complex = torch.polar(torch.ones_like(freqs), freqs)
#     return freqs_complex

# def apply_rotary_emb(x, freqs_complex, device):
#     x_complex = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1, 2))
#     freqs_complex = freqs_complex.unsqueeze(0).unsqueeze(2)
#     x_rotated = x_complex * freqs_complex
#     x_out = torch.view_as_real(x_rotated).reshape(*x.shape)
#     return x_out.type_as(x).to(device)

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.dropout = nn.Dropout(0.2)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):  # cos and sin are expected to be provided by the caller
        B, T, C = x.shape

        query = self.query(x)
        key = self.key(x)
        value = self.value(x)
    
    
        # query, key = apply_rotary_emb(query, key, freqs_cis=freqs_cis)
        
        wei = query @ key.transpose(-2,-1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        out = wei @ value  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out



class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out



class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.fc_1 = nn.Linear(n_embd, 4 * n_embd)
        self.fc_2 = nn.Linear(4 * n_embd, n_embd)
        self.droupout = nn.Dropout(0.2)

    def forward(self, x):
        x = new_gelu(self.fc_1(x))
        x = self.fc_2(x)
        x = self.droupout(x)
        return x


class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-8):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        return self.weight * self._norm(x.float()).type_as(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.rms_norm1 = RMSNorm(dim = n_embd)
        self.rms_norm2 = RMSNorm(dim = n_embd)

    def forward(self, x):
        x = x + self.sa(self.rms_norm1(x))
        x = x + self.ffwd(self.rms_norm2(x))
        return x

        
class GPT(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.rms_norm = RMSNorm(n_embd)
        self.linear_head = nn.Linear(n_embd, vocab_size)
    

    def forward(self, tokens, targets=None):
        B, T = tokens.shape
        
        # tokens and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding(tokens) # (B,T,C)
        pos_emb = self.position_embedding(torch.arange(T, device=device)) # (T,C)
       
        x = tok_emb + pos_emb # (B,T,C)

        for block in self.blocks:
            x = block(x)

        x = self.rms_norm(x) # (B,T,C)
        logits = self.linear_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx
    
model = GPT()

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

53.677261 M parameters


In [ ]:
# if you want to load pretrained model uncomment this block
# ckpt_path = os.path.join("saved/", 'ckpt.pt')
# checkpoint = torch.load(ckpt_path, map_location="cuda:0")
# state_dict = checkpoint['model']
# unwanted_prefix = '_orig_mod.'
# for k,v in list(state_dict.items()):
#     if k.startswith(unwanted_prefix):
#         state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
# model.load_state_dict(state_dict)


In [ ]:
model.to(device)
model = torch.compile(model)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [77]:
max_iters = 1000
eval_interval = 100
eval_iters = 100

In [78]:
# uncomment to train the model
# for iter in range(max_iters):
#     if iter % eval_interval == 0 or iter == max_iters - 1:
#         losses = estimate_loss()
#         print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
#     xb, yb = get_batch('train')
#     logits, loss = model(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.sum().backward()
#     optimizer.step()

In [79]:
# uncomment the block to save the model
# checkpoint = {
#     'model': model.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'iter_num': iter,
#     'val_loss': losses['val'],
# }
# print("saving checkpoint")
# torch.save(checkpoint, os.path.join("saved/", 'ckpt.pt'))
# print("saved checkpoint")

In [84]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
gen_text = decode(model.generate(context, max_new_tokens=256)[0].tolist())
print(gen_text)

Il le jeune prince . 'Ne pourriez -vous me branches , je pourrai la brise pour toi . - Tu ne peux pas y entrer aussi ? demanda -t-elle. - Mais oui , répondit le prince . Et ils sont pleines de vie Dès qu 'elle nage un parti , elle décida de voir son sang à faire , en fonte disparut et son chagrin . - Je vais trouver un enfant . » Mais la pleura toute sa fiancée , elle pied son chemin . Elle le changea et partit . Attends -moi le doigt et le jour , tout cachait ses courtisans . Après , elle prit la tête du prince , elle s 'assit et remit dans la partie de la chambre devinèrent que le vieux père . Chaque fois qu 'elle la princesse , elle prit la grenouille et la rendit dans l' intérieur , était sombre et lui dit : on ne dansa qu 'à rien voir d' autre . Elle passait . Elle lui donna alors seulement la jeta les lentilles dans une pièce , en sang et des plats d' or qu 'elle s 'incliner tout panier . Cependant la jeune fille de ses lèvres , elle lui montra dans la marmite du front , elle -mê

In [81]:
doc = nlp(gen_text)
displacy.render(doc, style="dep")

In [82]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Il il PRON PRON ROOT Xx True True
par par ADP ADP case xxx True True
son son DET DET det xxx True True
geste geste NOUN NOUN nmod xxxx True False
. . PUNCT PUNCT punct . False False
Elle lui PRON PRON nsubj Xxxx True True
prit prendre VERB VERB advcl xxxx True False
la le DET DET det xx True True
royale royale NOUN NOUN obj xxxx True False
, , PUNCT PUNCT punct , False False
sauta sauter VERB VERB ROOT xxxx True False
sur sur ADP ADP mark xxx True True
la le PRON PRON obj xx True True
mit mettre VERB VERB obl:arg xxx True False
en en ADP ADP case xx True True
tête tête NOUN NOUN obl:arg xxxx True False
. . PUNCT PUNCT punct . False False
Quand quand SCONJ SCONJ mark Xxxxx True True
la le DET DET det xx True True
grenouille grenouille NOUN NOUN nsubj xxxx True False
lui luire PRON PRON iobj xxx True True
apprit apprendre VERB VERB advcl xxxx True False
qu qu SCONJ SCONJ mark xx True False
' ' SCONJ SCONJ mark ' False False
il il PRON PRON nsubj xx True True
l l PRON PRON obj x True Fals